# Geocoding Addresses
## This notebook contains code to retrieve addresses from the articles dataframe and send them to Google's Geocoding service to receive lat/long coordinates for locating in a mapping service.
# Load articles data

In [ ]:
# Allows us to import packages that exist one level up in the file system
# See https://stackoverflow.com/questions/34478398
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path = [module_path] + sys.path

In [ ]:
from tagnews.utils import load_data as ld
import numpy as np
import pandas as pd
import pickle

In [ ]:
df = ld.load_data()

In [ ]:
df.head()

# Count total number of articles.

In [ ]:
df.loc[:, 'OEMC':].any(axis=1).sum()

# Count all the articles with addressses transcribed from the articles.

In [ ]:
df['locations'].apply(lambda x: bool(x)).sum()

In [ ]:
addr_list = df.locations[df.locations.apply(lambda x: bool(x))]

In [ ]:
addr_list.head()

# Count the total number of addresses transcribed from the articles to be geocoded.

In [ ]:
addr_list.apply(lambda x: len(x)).sum()

In [ ]:
count = 0
for i,j in addr_list.iteritems():
    if 'lat_long' not in j[0].keys():
        count += len(j)
count

# Run following 2 lines if making changes to lat_long.py. They allow this notebook to automatically update those changes for testing.

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

#  Please also note that you will need to supply an api_key from Google's Geocoding API site.
In the shell you launched this notebook from enter the following code with your geocode api key from Google:
```
        export GOOGLE_GEOCODE_API_KEY=api...
```


In [ ]:
import lat_long as ll
api_key = os.environ["GOOGLE_GEOCODE_API_KEY"]

# Main program to gather available locations data that does not yet have lat/long coordinates. 
### Set `test = True` to run smaller batches to prevent reaching Google's query limits too quickly.

In [ ]:
latlong_data = ll.get_lat_long(df, api_key, test=True)

In [ ]:
latlong_data.iloc[0]

In [ ]:
latlong_data.to_pickle('addr_geotag_list.pkl')

# Can see from running code block below that some of the queries didn't return results for a variety of reasons.

In [ ]:
count = 0
no_results = []
for i in latlong_data:
    for j in i:
        if 'lat_long' not in j.keys():
            count += 1
            no_results.append(j)
print('Number of addresses that didn\'t recieve lat/log coords: {}.'.format(count))
no_results

# Code block below can be run to see how the query works. You will need to supply your own api_key.

In [ ]:
from geopy.geocoders import GoogleV3
api_key = ''
g = GoogleV3(api_key = api_key, timeout = 10)

In [ ]:
latlong_data[0][1]['lat_long'][1][0]

In [ ]:
addr = addr_list[0][2]['cleaned text']
location = g.geocode(addr, components={'locality':'Chicago'})
location

# Code to construct URL for displaying lat/longs in Google Static Map

Was not able to get this work. Can get the map to output, but with no markers. Abandoning this for now.

In [ ]:
import mapper as m
import webbrowser

In [ ]:
mapURL = m.mapper(addr_list, api_key='')
webbrowser.open(mapURL)

In [ ]:
len(mapURL)

In [ ]:
baseURL = 'http://maps.googleapis.com/maps/api/staticmap?center=Chicago,IL&zoom=12&size=640x640&scale=2'
#markers = '&markers=size:mid&color:red'
#markers += '%7C' + str(latlong_data[0][1]['lat_long'][1][1]) + ',' + str(latlong_data[29][0]['lat_long'][1][1])
markers = '&markers=' + str(latlong_data[0][1]['lat_long'][1][0]) + ',' + str(latlong_data[0][1]['lat_long'][1][1])
mapURL = baseURL + markers + '&key='
webbrowser.open(mapURL)

In [ ]:
mapURL